<h1> Feedforward Fully-connected Neuralnetwork </h1>


<h3>Proto</h3>
단순하게 학습정도만 구현할것임. 버전관리하면서 기능 추가해볼것  

batch_size : size of mini-batch INT  
n_iteration : number of iterations INT  
n_neurons : number of neurons to hidden layer INT    
n_layers : number of hidden layers INT    
learning_rate : learning rate FLOAT    
hidden_activation : activation function. [ 'sigmoid', 'relu' ]   
output_activation : activation function for last layer. [ 'sigmoid', 'relu', 'softmax']  
cost_function : cost function [ 'crossentropy', 'mse' ]  
optimizer : gradient descent algorithm. [ 'minibatchSGD' ] 

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from operator import attrgetter

In [496]:
#preprocess

s = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
df = pd.read_csv(s,
                 header=None,
                 encoding='utf-8')

y = df.iloc[0:100,4].values
y = np.where(y == 'Iris-setosa', 0, 1)
X = df.iloc[0:100, [0, 2]].values

In [506]:
class FullyconnectedNN(object):
    def __init__(self, n_iteration = 1000, shuffle = True, batch_size = 32, n_neurons = 2, n_layers = 2, learning_rate = 0.1, hidden_activation = 'relu', output_activation = 'relu',
                 cost_function = 'mse', optimizer = 'minibatchSGD', random_state = 42):
        self.n_iteration = n_iteration
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.n_layers = n_layers
        self.n_neurons = n_neurons
        self.hidden_activation = hidden_activation
        self.learning_rate = learning_rate
        self.output_activation = output_activation
        self.cost_function_ = cost_function
        self.optimizer = optimizer
        self.random_state = random_state
        
    def fit(self, X, y):   
        def softmax(z):
            assert len(z.shape) == 2
            s = np.max(z, axis=1)
            s = s[:, np.newaxis]
            e_x = np.exp(z - s)
            div = np.sum(e_x, axis=1)
            div = div[:, np.newaxis]
            return e_x / div
        def mse(y_hat, y):
            return np.square(np.subtract(y_hat, y)).mean()
        def cross_entropy(y_hat, y):
            #add delta to prevent y_hat to be zero
            delta = 1e-7 
            return -np.sum(y*np.log(y_hat+delta))
        
        
        if isinstance(X, pd.DataFrame):
            X = X.to_numpy()
        assert X.shape[0] == len(y), "X and y don't match."
        rgen = np.random.RandomState(self.random_state)
        
        self.w_ = []
        self.b_ = []
        self.w_.append(abs(rgen.normal(loc = .0, scale = .01, size = X.shape[1] * self.n_neurons).reshape(X.shape[1], self.n_neurons)))
        self.b_.append(abs(rgen.normal(loc = .0, scale = .01, size = self.n_neurons)))
        for _ in range(self.n_layers - 1):
            self.w_.append(abs(rgen.normal(loc = .0, scale = .01, size = np.power(self.n_neurons,2)).reshape(self.n_neurons,self.n_neurons)))
            self.b_.append(abs(rgen.normal(loc = .0, scale = .01, size = self.n_neurons)))

        if self.cost_function_ in ['crossentropy']:
            self.w_.append(rgen.normal(loc = .0, scale = .01, size = len(Counter(y)) * self.n_neurons).reshape(len(Counter(y)), self.n_neurons))
            self.b_.append(rgen.normal(loc = .0, scale = .01, size = len(Counter(y))))
        else:
            self.w_.append(rgen.normal(loc = .0, scale = .01, size = self.n_neurons))
            self.b_.append(rgen.normal(loc = .0, scale = .01, size = 1))
        
        
        self.cost_ = []
        
        def create_minibatches(y):
            indices = np.arange(len(y))
            if self.shuffle:
                rgen.shuffle(indices)
            cut = [x for x in range(0, len(indices), self.batch_size)] + [len(indices)]
            mini_batches = []
            for i in range(1,len(cut)):
                mini_batches.append(indices[cut[i-1]:cut[i]])
            return mini_batches

        for _ in range(self.n_iteration):
            batches = create_minibatches(y)
            cost = []
            for batch in batches:
                X_batch = X[tuple(batch),:]
                y_batch = [y[int(batch_idx)] for batch_idx in batch]
                #forward-propagation
                self._y_hat = self.predict(X_batch)
                if self.cost_function_ == 'crossentropy':
                    cost.append(cross_entropy(self._y_hat, y_batch))
                if self.cost_function_ == 'mse':
                    cost.append(mse(self._y_hat, y_batch))
                
                #back-propagation
                self._d = []
                for i in range(1,self.n_layers + 1):
                    if self.cost_function_ == 'mse':
                        if i == 1:
                            self._d.append( ((self._y_hat - y_batch) * self._o_prime[-1]).mean(axis = 0) )
                            self.w_[-i] -= self.learning_rate * self._d[-1] * self._o[-1].mean(axis = 0)
                            self.b_[-i] -= self.learning_rate * self._d[-1]
                        else:
                            self._d.append( self._o_prime[-i].mean(axis = 0) *  np.dot(nn.w_[-(i-1)], nn._d[-1]) )
                            self.w_[-i] -= self.learning_rate * self._d[-1] * self._o[-i].mean(axis = 0)
                            self.b_[-i] -= self.learning_rate * self._d[-1]
                
                else:
                    self._d.append( self._o_prime[-i].mean(axis = 0) * np.dot(nn.w_[1], self._d[-1] ) )
                    self.w_[0] -= self.learning_rate * self._d[-1] * X.mean(axis = 0)
                    self.b_[0] -= self.learning_rate * self._d[-1]

            self.cost_.append(sum(cost)/len(cost))
        
        return self
            

        
    def predict(self, X):
        self._o = []
        self._o_prime = []
        sigmoid = lambda x:1/(1+np.exp(-x))
        
        for i in range(self.n_layers):
            if i == 0:
                if self.hidden_activation == 'relu':
                    self._o.append(np.maximum(np.matmul(X, self.w_[i]) + self.b_[i], 0))
                    self._o_prime.append(np.heaviside(np.matmul(X, self.w_[i]) + self.b_[i], 0))
                if self.hidden_activation == 'sigmoid':
                    self._o.append(sigmoid(np.matmul(X, self.w_[i]) + self.b_[i]))
                    self._o_prime.append( sigmoid(np.matmul(X, self.w_[i]) + self.b_[i]) * (1 - sigmoid(np.matmul(X, self.w_[i]) + self.b_[i])) )
            else:
                if self.hidden_activation == 'relu':
                    self._o.append(np.maximum(np.matmul(self._o[i-1], self.w_[i]) + self.b_[i], 0))
                    self._o_prime.append(np.heaviside(np.matmul(self._o[i-1], self.w_[i]) + self.b_[i], 0))
                    
                if self.hidden_activation == 'sigmoid':
                    self._o.append(sigmoid(np.matmul(self._o[i-1], self.w_[i]) + self.b_[i]))
                    self._o_prime.append( sigmoid(np.matmul(self._o[i-1], self.w_[i]) + self.b_[i]) * (1 - sigmoid(np.matmul(self._o[i-1], self.w_[i]) + self.b_[i])) )
                    
        if self.output_activation == 'relu':
            self._o_prime.append(np.heaviside(np.matmul(self._o[self.n_layers-1], self.w_[self.n_layers]) + self.b_[self.n_layers], 0))
            return np.maximum(np.matmul(self._o[self.n_layers-1], self.w_[self.n_layers]) + self.b_[self.n_layers], 0)
        
        if self.output_activation == 'sigmoid':
            self._o_prime.append( sigmoid(np.matmul(self._o[self.n_layers-1], self.w_[self.n_layers]) + self.b_[self.n_layers]) * (1 - sigmoid(np.matmul(self._o[self.n_layers-1], self.w_[self.n_layers]) + self.b_[self.n_layers])) )
            return sigmoid(np.matmul(self._o[self.n_layers-1], self.w_[self.n_layers]) + self.b_[self.n_layers])
        
        if self.output_activation == 'softmax':
            return softmax(np.matmul(self._o[self.n_layers-1], self.w_[self.n_layers]) + self.b_[self.n_layers])
        

In [518]:
nn = FullyconnectedNN(output_activation='sigmoid', hidden_activation='relu', learning_rate=0.01, n_layers= 5, n_neurons = 3)

In [519]:
nn.fit(X,y)

ValueError: operands could not be broadcast together with shapes (3,) (2,) 

In [520]:
X.mean(axis = 0)

array([5.471, 2.862])

In [501]:
self._d.append( self._o_prime[-i].mean(axis = 0) *  np.dot(nn.w_[-i], nn._d[-1]) )

NameError: name 'self' is not defined

In [503]:
nn._o_prime[-2].mean(axis = 0)

array([0.24998571, 0.24999193])

In [504]:
np.dot(nn.w_[-2],nn._d[-1])

array([[1.00031228e-06, 5.70691916e-07],
       [3.49109263e-07, 2.29445079e-07]])

In [505]:
nn.w_[-2]

array([[0.01579213, 0.00767435],
       [0.00469474, 0.0054256 ]])

In [379]:
np.dot(nn.w_[-1], nn._d[0])

array([-0.00016888,  0.00116183])

In [387]:
nn._o[-2].mean(axis = 0)

array([0.05331239, 0.06453249])